In [1]:
# IMPORT PACKAGES
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap, BoundaryNorm
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
import shapefile
import os

import netCDF4 as nc
from netCDF4 import Dataset
from shapely.geometry import Point, shape, box, mapping
from shapely.vectorized import contains
from shapely.strtree import STRtree

from rasterio.coords import BoundingBox
from rasterio.mask import mask
from rasterio.plot import show
from rasterio.features import shapes

import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np
from scipy.ndimage import sobel

In [2]:
# DEFINE INPUTS FOR FUNCTION
masterlist = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/ALL_STATIONS_FINAL_REVISED.csv'

full_df = pd.read_csv(masterlist)
station_num = full_df['grdc_no']
full_df['average_slope'] = np.nan

print(f"Loaded {len(station_num)} stations.")

Loaded 553 stations.


In [3]:
# for i in range(0):
i = 0
data = full_df.iloc[i]
number = data['grdc_no']
region = data['wmo_reg']
river = data['river']
name = data['station']
lat = data['lat']
lon = data['long']
area = data['area']
altitude = data['altitude']
shp_log = data['shapefile_code']

tif = '/global/scratch/users/arvalcarcel/CSMUB/DATA/aridity_index.tif'
# if region == 1:
#   dem = '/global/scratch/users/arvalcarcel/CSMUB/DATA/DEM/hyd_af_dem_30s.tif'
# elif region == 2:
#   dem = '/global/scratch/users/arvalcarcel/CSMUB/DATA/DEM/hyd_as_dem_30s.tif'
# elif region == 3:
#   dem = '/global/scratch/users/arvalcarcel/CSMUB/DATA/DEM/hyd_sa_dem_30s.tif'
# elif region == 4:
#   dem = '/global/scratch/users/arvalcarcel/CSMUB/DATA/DEM/hyd_na_dem_30s.tif'
# elif region == 5:
#   dem = '/global/scratch/users/arvalcarcel/CSMUB/DATA/DEM/hyd_au_dem_30s.tif'
# elif region == 6:
#   dem = '/global/scratch/users/arvalcarcel/CSMUB/DATA/DEM/hyd_eu_dem_30s.tif'
# # output_tif = f'/global/scratch/users/arvalcarcel/CSMUB/DATA/DEM/STATIONS/{number}_dem.tif'

In [4]:
# Read the shapefiles
shapefile1 = f'/global/scratch/users/arvalcarcel/CSMUB/DATA/DEM/STATIONS/{number}/{number}.shp' # delineated shapefile
shapefile2 = f'/global/scratch/users/arvalcarcel/CSMUB/DATA/SHAPEFILES/{number}/{number}.shp' # GRDC shapefile

if shp_log == 1:
    shapefile = shapefile1

elif shp_log == 2:
    shapefile = shapefile2

In [5]:
def calculate_average_aridity(tif_path, shapefile_path):
    # Load shapefile
    gdf = gpd.read_file(shapefile_path)
    print("CRS of shapefile:", gdf.crs)
    shapes = gdf.geometry.values

    # Load DEM and mask it with the shapefile
    with rasterio.open(tif_path) as src:
        aridity_data, transform = mask(src, shapes, crop=True)
        aridity_data = aridity_data[0].astype('float32')  # Convert to float

        if src.nodata is not None:
            aridity_data[aridity_data == src.nodata] = np.nan

    # Average slope
    aridity_index = np.nanmean(aridity_data)
    return aridity_index

In [6]:
# Example usage
aridity_index = calculate_average_aridity(tif, shapefile)
print(f"Average aridity index within the area: {aridity_index:.2f}")

CRS of shapefile: EPSG:4326
Average aridity index within the area: 907.28


In [9]:
# Merge total_df with landcover_df on GRDC_No
# Store the average slope in the DataFrame
full_df.at[i, 'avg_aridity'] = aridity_index

print(full_df.head())

   Unnamed: 0  grdc_no  wmo_reg  sub_reg           river  \
0           0  1159100        1     1591    ORANGE RIVER   
1           1  1159103        1     1591    ORANGE RIVER   
2           2  1159105        1     1591    ORANGE RIVER   
3           3  1159125        1     1591  RENOSTERRIVIER   
4           4  1159130        1     1591             SAK   

                      station country        lat       long      area  ...  \
0       VIOOLSDRIF (27811003)      ZA -28.757990  17.721591  866486.0  ...   
1               PELLA MISSION      ZA -28.963403  19.151922  859920.0  ...   
2              SENDELINGSDRIF      ZA -28.075833  16.898333  985370.0  ...   
3                 LEEUWENKUIL      ZA -31.437160  20.474169    8938.0  ...   
4  HOL PADS LEEGTE (27814016)      ZA -30.474712  20.519176   40073.0  ...   

   r_volume_yr  r_height_yr  shapefile_code        r2  NaNs  average_slope  \
0     7.928970     9.150719               1  0.323816     6            NaN   
1     4.162973

In [ ]:
# full_df.to_csv(masterlist, index=False)
# print(pd.read_csv(masterlist))